<center>
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/module_1/images/SN_web_lightmode.png" width="300"> 
</center>


<h1>Data Wrangling with dplyr</h1>

Estimated time needed: **60** minutes


## Lab Overview:

In this lab, you will focus on wrangling the Seoul bike-sharing demand historical dataset. This is the core dataset to build a predictive model later.

It contains the following columns:

- `DATE` : Year-month-day
- `RENTED BIKE COUNT`- Count of bikes rented at each hour
- `HOUR`- Hour of he day
- `TEMPERATURE` - Temperature in Celsius
- `HUMIDITY` - Unit is `%`
- `WINDSPEED` - Unit is `m/s`
- `VISIBILITY` - Multiplied by 10m
- `DEW POINT TEMERATURE` - The temperature to which the air would have to cool down in order to reach saturation, unit is Celsius
- `SOLAR RADIATION` - MJ/m2
- `RAINFALL` - mm
- `SNOWFALL` - cm
- `SEASONS` - Winter, Spring, Summer, Autumn
- `HOLIDAY` - Holiday/No holiday
- `FUNCTIONAL DAY` - NoFunc(Non Functional Hours), Fun(Functional hours)


For this dataset, you will be asked to use `tidyverse` to perform the following data wrangling tasks:

- **TASK: Detect and handle missing values**
- **TASK: Create indicator (dummy) variables for categorical variables**
- **TASK: Normalize data**

Let's start!


First import the necessary library for this data wrangling task:


In [1]:
# Check if you need to install the `tidyverse` library
# require("tidyverse")
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──
✔ ggplot2 3.3.0     ✔ purrr   0.3.4
✔ tibble  3.0.1     ✔ dplyr   0.8.5
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Then load the bike-sharing system data from the csv processed in the previous lab:


In [2]:
bike_sharing_df <- read_csv("raw_seoul_bike_sharing.csv")

Parsed with column specification:
cols(
  DATE = col_character(),
  RENTED_BIKE_COUNT = col_double(),
  HOUR = col_double(),
  TEMPERATURE = col_double(),
  HUMIDITY = col_double(),
  WIND_SPEED = col_double(),
  VISIBILITY = col_double(),
  DEW_POINT_TEMPERATURE = col_double(),
  SOLAR_RADIATION = col_double(),
  RAINFALL = col_double(),
  SNOWFALL = col_double(),
  SEASONS = col_character(),
  HOLIDAY = col_character(),
  FUNCTIONING_DAY = col_character()
)


In [ ]:
# Or you may read it from here again
# url <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/raw_seoul_bike_sharing.csv"
# Notice some column names in the raw datasets are not standalized if you haven't done them properly in the previous lab


First take a quick look at the dataset:


In [3]:
summary(bike_sharing_df)
dim(bike_sharing_df)

     DATE           RENTED_BIKE_COUNT      HOUR        TEMPERATURE    
 Length:8760        Min.   :   2.0    Min.   : 0.00   Min.   :-17.80  
 Class :character   1st Qu.: 214.0    1st Qu.: 5.75   1st Qu.:  3.40  
 Mode  :character   Median : 542.0    Median :11.50   Median : 13.70  
                    Mean   : 729.2    Mean   :11.50   Mean   : 12.87  
                    3rd Qu.:1084.0    3rd Qu.:17.25   3rd Qu.: 22.50  
                    Max.   :3556.0    Max.   :23.00   Max.   : 39.40  
                    NA's   :295                       NA's   :11      
    HUMIDITY       WIND_SPEED      VISIBILITY   DEW_POINT_TEMPERATURE
 Min.   : 0.00   Min.   :0.000   Min.   :  27   Min.   :-30.600      
 1st Qu.:42.00   1st Qu.:0.900   1st Qu.: 940   1st Qu.: -4.700      
 Median :57.00   Median :1.500   Median :1698   Median :  5.100      
 Mean   :58.23   Mean   :1.725   Mean   :1437   Mean   :  4.074      
 3rd Qu.:74.00   3rd Qu.:2.300   3rd Qu.:2000   3rd Qu.: 14.800      
 Max.   :98.

[1] 8760   14

From the summary, we can observe that:

Columns `RENTED_BIKE_COUNT`, `TEMPERATURE`, `HUMIDITY`, `WIND_SPEED`, `VISIBILITY`, `DEW_POINT_TEMPERATURE`, `SOLAR_RADIATION`, `RAINFALL`, `SNOWFALL` are numerical variables/columns and require normalization. Moreover, `RENTED_BIKE_COUNT` and `TEMPERATURE` have some missing values (NA's) that need to be handled properly.

`SEASONS`, `HOLIDAY`, `FUNCTIONING_DAY` are categorical variables which need to be converted into indicator columns or dummy variables.
Also, `HOUR` is read as a numerical variable but it is in fact a categorical variable with levels ranging from 0 to 23.

Now that you have some basic ideas about how to process this bike-sharing demand dataset, let's start working on it!


# TASK: Detect and handle missing values


 The `RENTED_BIKE_COUNT` column has about 295 missing values, and `TEMPERATURE` has about 11 missing values. Those missing values could be caused by not being recorded, or from malfunctioning bike-sharing systems or weather sensor networks. In any cases, the identified missing values have to be properly handled.


Let's first handle missing values in `RENTED_BIKE_COUNT` column:


Considering `RENTED_BIKE_COUNT` is the response variable/dependent variable, i.e., we want to predict the `RENTED_BIKE_COUNT` using other predictor/independent variables later, and we normally can not allow missing values for the response variable, so missing values for response variable must be either dropped or imputed properly. 

We can see that `RENTED_BIKE_COUNT` only has about 3% missing values (295 / 8760). As such, you can safely drop any rows whose `RENTED_BIKE_COUNT` has missing values.


_TODO:_ Drop rows with missing values in the `RENTED_BIKE_COUNT` column


In [4]:
# Drop rows with `RENTED_BIKE_COUNT` column == NA
bike_sharing_df <- bike_sharing_df %>% drop_na(RENTED_BIKE_COUNT)

In [5]:
# Print the dataset dimension again after those rows are dropped
dim(bike_sharing_df)

[1] 8465   14

Now that you have handled  missing values in the `RENTED_BIKE_COUNT` variable, let's continue processing missing values for the `TEMPERATURE` column.


Unlike the `RENTED_BIKE_COUNT` variable, `TEMPERATURE` is not a response variable. However, it is still an important predictor variable - as you could imagine, there may be a positve correlation between `TEMPERATURE` and `RENTED_BIKE_COUNT`. For example, in winter time with lower temperatures, people may not want to ride a bike, while in summer with nicer weather, they are more likely to rent a bike.


How do we handle missing values for `TEMPERATURE`? We could simply remove the rows but it's better to impute them because `TEMPERATURE` should be relatively easy and reliable to estimate statistically.


Let's first take a look at the missing values in the TEMPERATURE column.


In [6]:
bike_sharing_df %>% 
                filter(is.na(TEMPERATURE))

DATE,RENTED_BIKE_COUNT,HOUR,TEMPERATURE,HUMIDITY,WIND_SPEED,VISIBILITY,DEW_POINT_TEMPERATURE,SOLAR_RADIATION,RAINFALL,SNOWFALL,SEASONS,HOLIDAY,FUNCTIONING_DAY
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
07/06/2018,3221,18,NA,57,2.7,1217,16.4,0.96,0.0,0,Summer,No Holiday,Yes
12/06/2018,1246,14,NA,45,2.2,1961,12.7,1.39,0.0,0,Summer,No Holiday,Yes
13/06/2018,2664,17,NA,57,3.3,919,16.4,0.87,0.0,0,Summer,No Holiday,Yes
17/06/2018,2330,17,NA,58,3.3,865,16.7,0.66,0.0,0,Summer,No Holiday,Yes
20/06/2018,2741,19,NA,61,2.7,1236,17.5,0.60,0.0,0,Summer,No Holiday,Yes
30/06/2018,1144,13,NA,87,1.7,390,23.2,0.71,3.5,0,Summer,No Holiday,Yes
05/07/2018,827,10,NA,75,1.1,1028,20.8,1.22,0.0,0,Summer,No Holiday,Yes
11/07/2018,634,9,NA,96,0.6,450,24.9,0.41,0.0,0,Summer,No Holiday,Yes
12/07/2018,593,6,NA,93,1.1,852,24.3,0.01,0.0,0,Summer,No Holiday,Yes


It seems that all of the missing values for `TEMPERATURE` are found in `SEASONS == Summer`, so it is reasonable to impute those missing values with the summer average temperature.


_TODO:_ Impute missing values for the TEMPERATURE column using its mean value.


In [11]:
# Calculate the summer average temperature
summer_data <- bike_sharing_df %>%
  filter(SEASONS == "Summer")

summer_avg_temp = round(mean(summer_data$TEMPERATURE, na.rm = TRUE),2)
summer_avg_temp

[1] 26.59

In [12]:
# Impute missing values for TEMPERATURE column with summer average temperature
bike_sharing_df = bike_sharing_df %>% 
                replace_na(list(TEMPERATURE = summer_avg_temp))

In [13]:
# Print the summary of the dataset again to make sure no missing values in all columns
summary(bike_sharing_df)

     DATE           RENTED_BIKE_COUNT      HOUR        TEMPERATURE    
 Length:8465        Min.   :   2.0    Min.   : 0.00   Min.   :-17.80  
 Class :character   1st Qu.: 214.0    1st Qu.: 6.00   1st Qu.:  3.00  
 Mode  :character   Median : 542.0    Median :12.00   Median : 13.50  
                    Mean   : 729.2    Mean   :11.51   Mean   : 12.77  
                    3rd Qu.:1084.0    3rd Qu.:18.00   3rd Qu.: 22.70  
                    Max.   :3556.0    Max.   :23.00   Max.   : 39.40  
    HUMIDITY       WIND_SPEED      VISIBILITY   DEW_POINT_TEMPERATURE
 Min.   : 0.00   Min.   :0.000   Min.   :  27   Min.   :-30.600      
 1st Qu.:42.00   1st Qu.:0.900   1st Qu.: 935   1st Qu.: -5.100      
 Median :57.00   Median :1.500   Median :1690   Median :  4.700      
 Mean   :58.15   Mean   :1.726   Mean   :1434   Mean   :  3.945      
 3rd Qu.:74.00   3rd Qu.:2.300   3rd Qu.:2000   3rd Qu.: 15.200      
 Max.   :98.00   Max.   :7.400   Max.   :2000   Max.   : 27.200      
 SOLAR_RADIAT

In [14]:
# Save the dataset as `seoul_bike_sharing.csv`
write.csv(bike_sharing_df, "seoul_bike_sharing.csv", row.names = FALSE)

# TASK: Create indicator (dummy) variables for categorical variables


Regression models can not process categorical variables directly, thus we need to convert them into indicator variables. 


In the bike-sharing demand dataset, `SEASONS`, `HOLIDAY`, `FUNCTIONING_DAY` are categorical variables. 
Also, `HOUR` is read as a numerical variable but it is in fact a categorical variable with levels ranged from 0 to 23.


_TODO:_ Convert `HOUR` column from numeric into character first:


In [15]:
# Using mutate() function to convert HOUR column into character type
bike_sharing_df <- bike_sharing_df %>%
              mutate(HOUR = as.character(HOUR))

In [16]:
summary(bike_sharing_df$HOUR)

   Length     Class      Mode 
     8465 character character 

`SEASONS`, `HOLIDAY`, `FUNCTIONING_DAY`,  `HOUR` are all character columns now and are ready to be converted into indicator variables.

For example, `SEASONS` has four categorical values: `Spring`, `Summer`, `Autumn`, `Winter`. We thus need to create four indicator/dummy variables `Spring`, `Summer`, `Autumn`, and `Winter` which only have the value 0 or 1.

So, given a data entry with the value `Spring` in the `SEASONS` column, the values for the four new columns `Spring`, `Summer`, `Autumn`, and `Winter` will be set to 1 for `Spring` and 0 for the others:

|Spring|Summer|Autumn|Winter|
|----- |------|------|------|
|     1|     0|     0|     0| 


_TODO:_ Convert `SEASONS`, `HOLIDAY`, `FUNCTIONING_DAY`, and `HOUR` columns into indicator columns.

Note that if `FUNCTIONING_DAY` only contains one categorical value after missing values removal, then you don't need to convert it to an indicator column.


In [18]:
counts <- table(bike_sharing_df$FUNCTIONING_DAY)
print(counts)


 Yes 
8465 


In [22]:
# Convert SEASONS, HOLIDAY, FUNCTIONING_DAY, and HOUR columns into indicator columns.

bike_sharing_df = bike_sharing_df %>%
                  mutate(dummy = 1) %>% # column with single value
                  spread(
                    key = SEASONS, # column to spread
                    value = dummy,
                    fill = 0)

bike_sharing_df = bike_sharing_df %>%
                  mutate(dummy = 1) %>% # column with single value
                  spread(
                    key = HOLIDAY, # column to spread
                    value = dummy,
                    fill = 0)

bike_sharing_df = bike_sharing_df %>%
                  mutate(dummy = 1) %>% # column with single value
                  spread(
                    key = HOUR, # column to spread
                    value = dummy,
                    fill = 0)

In [27]:
head(bike_sharing_df)

DATE,RENTED_BIKE_COUNT,TEMPERATURE,HUMIDITY,WIND_SPEED,VISIBILITY,DEW_POINT_TEMPERATURE,SOLAR_RADIATION,RAINFALL,SNOWFALL,⋯,21,22,23,3,4,5,6,7,8,9
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
01/12/2017,254,-5.2,37,2.2,2000,-17.6,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
01/12/2017,204,-5.5,38,0.8,2000,-17.6,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
01/12/2017,173,-6.0,39,1.0,2000,-17.7,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
01/12/2017,107,-6.2,40,0.9,2000,-17.6,0,0,0,⋯,0,0,0,1,0,0,0,0,0,0
01/12/2017,78,-6.0,36,2.3,2000,-18.6,0,0,0,⋯,0,0,0,0,1,0,0,0,0,0
01/12/2017,100,-6.4,37,1.5,2000,-18.7,0,0,0,⋯,0,0,0,0,0,1,0,0,0,0


In [28]:
# Print the dataset summary again to make sure the indicator columns are created properly
summary(bike_sharing_df)

     DATE           RENTED_BIKE_COUNT  TEMPERATURE        HUMIDITY    
 Length:8465        Min.   :   2.0    Min.   :-17.80   Min.   : 0.00  
   WIND_SPEED      VISIBILITY   DEW_POINT_TEMPERATURE SOLAR_RADIATION 
 Min.   :0.000   Min.   :  27   Min.   :-30.600       Min.   :0.0000  
    RAINFALL          SNOWFALL       FUNCTIONING_DAY        Autumn      
 Min.   : 0.0000   Min.   :0.00000   Length:8465        Min.   :0.0000  
     Spring           Summer           Winter          Holiday      
 Min.   :0.0000   Min.   :0.0000   Min.   :0.0000   Min.   :0.0000  
   No Holiday           0                 1                 10        
 Min.   :0.0000   Min.   :0.00000   Min.   :0.00000   Min.   :0.0000  
       11               12               13               14        
 Min.   :0.0000   Min.   :0.0000   Min.   :0.0000   Min.   :0.0000  
       15               16               17               18        
 Min.   :0.0000   Min.   :0.0000   Min.   :0.0000   Min.   :0.0000  
       19     

In [29]:
# Save the dataset as `seoul_bike_sharing_converted.csv`
write.csv(bike_sharing_df, "seoul_bike_sharing_converted.csv", row.names = FALSE)

# TASK: Normalize data


Columns `RENTED_BIKE_COUNT`, `TEMPERATURE`, `HUMIDITY`, `WIND_SPEED`, `VISIBILITY`, `DEW_POINT_TEMPERATURE`, `SOLAR_RADIATION`, `RAINFALL`, `SNOWFALL` are numerical variables/columns with different value units and range. Columns with large values may adversely influence (bias) the predictive models and degrade model accuracy. Thus, we need to perform normalization on these numeric columns to transfer them into a similar range.


In this project, you are asked to use Min-max normalization: 

**Min-max** rescales each value in a column by first subtracting the minimum value of the column from each value, and then divides the result by the difference between the maximum and minimum values of the column. So the column gets re-scaled such that the minimum becomes 0 and the maximum becomes 1.

$$x_{new} = \frac{x_{old} - x_{min}}{x_{max} - x_{min}}$$


_TODO:_ Apply min-max normalization on `RENTED_BIKE_COUNT`, `TEMPERATURE`, `HUMIDITY`, `WIND_SPEED`, `VISIBILITY`, `DEW_POINT_TEMPERATURE`, `SOLAR_RADIATION`, `RAINFALL`, `SNOWFALL`


In [18]:
columns <- c("RENTED_BIKE_COUNT", "TEMPERATURE", "HUMIDITY", 
             "WIND_SPEED", "VISIBILITY", "DEW_POINT_TEMPERATURE",
             "SOLAR_RADIATION", "RAINFALL", "SNOWFALL")

# Normalize each column in the dataframe
for (col_name in columns) {
  # Access the column from the dataframe
  column <- bike_sharing_df[[col_name]]
  
  # Normalize the column using min-max normalization
  bike_sharing_df[[col_name]] <- (column - min(column)) / (max(column) - min(column))
}

In [25]:
print(summary(bike_sharing_df$RENTED_BIKE_COUNT))

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
0.00000 0.05965 0.15194 0.20460 0.30445 1.00000 


In [26]:
# Print the summary of the dataset again to make sure the numeric columns range between 0 and 1

for (column_name in columns){
    column <- bike_sharing_df[[column_name]]
    print(column_name)
    print(summary(column))
}

[1] "RENTED_BIKE_COUNT"
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
0.00000 0.05965 0.15194 0.20460 0.30445 1.00000 
[1] "TEMPERATURE"
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.0000  0.3636  0.5472  0.5345  0.7080  1.0000 
[1] "HUMIDITY"
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.0000  0.4286  0.5816  0.5933  0.7551  1.0000 
[1] "WIND_SPEED"
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.0000  0.1216  0.2027  0.2332  0.3108  1.0000 
[1] "VISIBILITY"
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.0000  0.4602  0.8429  0.7131  1.0000  1.0000 
[1] "DEW_POINT_TEMPERATURE"
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.0000  0.4412  0.6107  0.5977  0.7924  1.0000 
[1] "SOLAR_RADIATION"
    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
0.000000 0.000000 0.002841 0.161326 0.264205 1.000000 
[1] "RAINFALL"
    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
0.000000 0.000000 0.000000 0.004261 0.000000 1.000000 
[1] "SNOWFALL"
    Min.  1st Qu.   M

In [27]:
# Save the dataset as `seoul_bike_sharing_converted_normalized.csv`
write.csv(bike_sharing_df, "seoul_bike_sharing_converted_normalized.csv", row.names = FALSE)

## Standardize the column names again for the new datasets


Since you have added many new indicator variables, you need to standardize their column names again by using the following code:


In [28]:
# Dataset list
dataset_list <- c('seoul_bike_sharing.csv', 'seoul_bike_sharing_converted.csv', 'seoul_bike_sharing_converted_normalized.csv')

for (dataset_name in dataset_list){
    # Read dataset
    dataset <- read_csv(dataset_name)
    # Standardized its columns:
    # Convert all columns names to uppercase
    names(dataset) <- toupper(names(dataset))
    # Replace any white space separators by underscore, using str_replace_all function
    names(dataset) <- str_replace_all(names(dataset), " ", "_")
    # Save the dataset back
    write.csv(dataset, dataset_name, row.names=FALSE)
}

Parsed with column specification:
cols(
  DATE = col_character(),
  RENTED_BIKE_COUNT = col_double(),
  HOUR = col_double(),
  TEMPERATURE = col_double(),
  HUMIDITY = col_double(),
  WIND_SPEED = col_double(),
  VISIBILITY = col_double(),
  DEW_POINT_TEMPERATURE = col_double(),
  SOLAR_RADIATION = col_double(),
  RAINFALL = col_double(),
  SNOWFALL = col_double(),
  SEASONS = col_character(),
  HOLIDAY = col_character(),
  FUNCTIONING_DAY = col_character()
)
Parsed with column specification:
cols(
  .default = col_double(),
  DATE = col_character(),
  FUNCTIONING_DAY = col_character()
)
See spec(...) for full column specifications.
Parsed with column specification:
cols(
  .default = col_double(),
  DATE = col_character(),
  FUNCTIONING_DAY = col_character()
)
See spec(...) for full column specifications.


# Next Steps


Great! Now that you have processed all of the necessary datasets, you are ready to perform exploratory data analysis to get some inital insights from them.


## Authors

<a href="https://www.linkedin.com/in/yan-luo-96288783/" target="_blank">Yan Luo</a>


### Other Contributors

Jeff Grossman


## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description           |
| ----------------- | ------- | ---------- | ---------------------------- |
| 2021-04-08        | 1.0     | Yan        | Initial version created      |
|                   |         |            |                              |
|                   |         |            |                              |

## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
